In [131]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product 
import pandas as pd
from sklearn.cluster import KMeans
from time import time
import os
import datetime
%matplotlib qt

In [132]:
def run_sudoku(E,I,eps = 0.01,time = 300):
    n = 81

    space = np.zeros((n,int(time/eps)))
    
    metric = np.inf*np.ones(9)

    def U(x,gamma=1,I=1.01):
        tif = -1/gamma*np.log(1-(gamma/I))
        return I/gamma*(1-np.exp(-tif*x))

    def U_inv(x,gamma=1,I=1.01):
        tif = -1/gamma*np.log(1-(gamma/I))
        return 1/tif*np.log(1/(1-(gamma/I)*x))

    def H(x,strength,gamma=1,I=1.01):
        temp = np.clip(U(x,gamma=gamma,I=I)+strength,0,1)
        return U_inv(temp,gamma=gamma,I=I)

    inh = np.load("sudoku.npy")
    exi = 1-inh
    np.fill_diagonal(exi,0.0)
    
    state = np.random.normal(size=n)%1

    for i in range(int(time/eps)):

        fire_time = np.min(1-state)   

        state = state + fire_time

        new_state = (state+eps)

        m = 0
        while np.sum(new_state>1)>0:
            if m>1000:
                return False,state,metric
            m+=1

            fire = new_state>1

            new_state[fire] = 0.0

            coupling_exi = E*(np.matmul(exi,fire)>0)
            coupling_inh = I*np.matmul(inh,fire)

            total = coupling_exi-coupling_inh

            state = new_state
            new_state = np.clip(H(state,total),a_min=0,a_max=None)
            
            if np.isnan(new_state.sum()):
                return False,state,metric

        state = new_state
        space[:,i] = state
    
    
    df = pd.DataFrame(space[:,-100:])
    model = KMeans(n_clusters=9)
    pred = model.fit_predict(df)
    
    for i in range(9):
        temp = np.arange(0,81,1,dtype=np.int32)[pred==i]
        sum_ = 0
        for j,k in product(temp,temp):
            sum_ += inh[j,k]
        metric[i]=sum_
    
    return True,space,metric

In [136]:
t0 = time()
res = 30

E = 1/10**np.linspace(2,3,res)
I = 1/10**np.linspace(2,4,res)
replicate = 3

succ_mat = np.zeros((E.shape[0],I.shape[0],replicate))
metric_mat = np.zeros((E.shape[0],I.shape[0],replicate))
time_mat = np.zeros((E.shape[0],I.shape[0],replicate))

folder = "S "+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
if not os.path.exists("search"):
    os.mkdir(folder)

pred_time = res**2*max(2,time_mat.mean())*replicate
print("Predicted Time:",pred_time,"secs")
    
count = 0
for x,e in enumerate(E):
    for y,i in enumerate(I):
        for z in range(replicate):
            t = time()
            success,space,metric = run_sudoku(e,i)
            time_mat[x,y,z] = time()-t
            succ_mat[x,y,z] = success
            metric_mat[x,y,z] = np.sum(metric)
            if np.sum(metric)<120:
                dirname =folder+"/"+"E "+str(format(e,".2e"))+" I "+str(format(i,".2e"))+" Replicate "+str(z)
                os.mkdir(dirname)
                np.save(dirname+'/space',space)
                np.save(dirname+'/metric',metric)
    if count%100==0:
        print('Progress',round((x+y)/(E.shape[0]*I.shape[0]),1),"%")
    count+=1
print("Total Time:",time()-t0,"secs")

Predicted Time: 5400 secs


/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-

/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-

/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)


Progress 0.0 %


...

/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-

/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-

/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-

/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return_n_iter=True)
/home/iiser/.conda/envs/venv/lib/python3.6/site-

KeyboardInterrupt: 

In [122]:
sns.heatmap(metric_mat<100,xticklabels=[str(format(e,".2e")) for e in E],yticklabels=[str(format(i,".2e")) for i in I])

In [ ]:
metric_mat.mean(axis=2)

In [112]:
E[23],I[21]

(0.0013257113655901094, 0.0019306977288832496)

In [ ]:
np.ma

In [137]:
E

array([0.01      , 0.00923671, 0.00853168, 0.00788046, 0.00727895,
       0.00672336, 0.00621017, 0.00573615, 0.00529832, 0.0048939 ,
       0.00452035, 0.00417532, 0.00385662, 0.00356225, 0.00329034,
       0.0030392 , 0.00280722, 0.00259294, 0.00239503, 0.00221222,
       0.00204336, 0.00188739, 0.00174333, 0.00161026, 0.00148735,
       0.00137382, 0.00126896, 0.0011721 , 0.00108264, 0.001     ])